# Titanic Baseline Model

Simple baseline using basic features and RandomForest classifier.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings('ignore')

# Load data
train_df = pd.read_csv('/home/data/train.csv')
test_df = pd.read_csv('/home/data/test.csv')

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print("\nTrain columns:", train_df.columns.tolist())
print("\nFirst few rows:")
train_df.head()

Train shape: (891, 12)
Test shape: (418, 11)

Train columns: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

First few rows:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# Basic EDA
print("Missing values in train:")
print(train_df.isnull().sum())
print("\nMissing values in test:")
print(test_df.isnull().sum())

print("\nSurvival rate:", train_df['Survived'].mean())
print("\nSurvival by gender:")
print(train_df.groupby('Sex')['Survived'].mean())
print("\nSurvival by class:")
print(train_df.groupby('Pclass')['Survived'].mean())

Missing values in train:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Missing values in test:
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Survival rate: 0.3838383838383838

Survival by gender:
Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

Survival by class:
Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64


In [3]:
# Feature engineering - simple baseline features
def create_features(df):
    df = df.copy()
    
    # Extract title from name
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    
    # Simplify titles
    title_mapping = {
        'Mr': 'Mr', 'Mrs': 'Mrs', 'Miss': 'Miss', 'Master': 'Master',
        'Dr': 'Other', 'Rev': 'Other', 'Col': 'Other', 'Major': 'Other',
        'Mlle': 'Miss', 'Countess': 'Other', 'Ms': 'Miss', 'Lady': 'Other',
        'Jonkheer': 'Other', 'Don': 'Other', 'Dona': 'Other', 'Mme': 'Mrs',
        'Capt': 'Other', 'Sir': 'Other'
    }
    df['Title'] = df['Title'].map(title_mapping)
    
    # Family size
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    
    # Is alone
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    
    # Age groups
    df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 12, 18, 35, 60, 100], 
                           labels=['Child', 'Teen', 'Adult', 'MiddleAge', 'Senior'])
    
    # Fare per person
    df['FarePerPerson'] = df['Fare'] / df['FamilySize']
    
    # Extract deck from cabin
    df['Deck'] = df['Cabin'].str[0]
    df['Deck'] = df['Deck'].fillna('Unknown')
    
    return df

# Create features for both datasets
train_feat = create_features(train_df)
test_feat = create_features(test_df)

print("Features created successfully")
print("New columns:", [col for col in train_feat.columns if col not in train_df.columns])

Features created successfully
New columns: ['Title', 'FamilySize', 'IsAlone', 'AgeGroup', 'FarePerPerson', 'Deck']


In [4]:
# Define feature columns
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'FarePerPerson']
categorical_features = ['Pclass', 'Sex', 'Embarked', 'Title', 'IsAlone', 'AgeGroup', 'Deck']

# Prepare data
X = train_feat[numeric_features + categorical_features]
y = train_feat['Survived']
X_test = test_feat[numeric_features + categorical_features]

print("Training features shape:", X.shape)
print("Test features shape:", X_test.shape)

Training features shape: (891, 13)
Test features shape: (418, 13)


In [5]:
# Create preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('encoder', LabelEncoder())
])

# Need to handle LabelEncoder for multiple columns
from sklearn.preprocessing import OneHotEncoder

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create full pipeline
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
])

print("Pipeline created")

Pipeline created


In [6]:
# Cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy', n_jobs=-1)

print(f"CV Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f"Individual fold scores: {cv_scores}")

CV Accuracy: 0.8170 ± 0.0107
Individual fold scores: [0.83240223 0.80898876 0.80337079 0.81460674 0.8258427 ]


In [7]:
# Train on full data and predict
clf.fit(X, y)

# Make predictions
test_predictions = clf.predict(X_test)

# Create submission
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': test_predictions
})

print("Submission shape:", submission.shape)
print("\nFirst 10 predictions:")
print(submission.head(10))

# Save submission
submission.to_csv('/home/submission/submission.csv', index=False)
print("\nSubmission saved to /home/submission/submission.csv")

Submission shape: (418, 2)

First 10 predictions:
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         1
4          896         1
5          897         0
6          898         0
7          899         0
8          900         1
9          901         0

Submission saved to /home/submission/submission.csv


In [8]:
# Check prediction distribution
print("Prediction distribution:")
print(submission['Survived'].value_counts())
print(f"Survival rate in predictions: {submission['Survived'].mean():.3f}")
print(f"Survival rate in training: {y.mean():.3f}")

Prediction distribution:
Survived
0    263
1    155
Name: count, dtype: int64
Survival rate in predictions: 0.371
Survival rate in training: 0.384
